## 종만북 chapter 6. 무식하게 풀기

- Brute-Force, Exhaustive Search, 완전탐색.
    - 모든 가능한 경우의 수를 일일이 나열하면서 답을 찾는 방법
    - 컴퓨터의 장점을 가장 잘 이용하는 방법.
    - 입력의 크기가 작은경우 매우 쉽게 풀 수 있다.

## 재귀 호출과 완전 탐색

### 재귀 호출


수행할 작업을 유사한 형태의 여러조각으로 쪼갠 뒤 그 중 한 조각을 수행, 나머지를 자기 자신을 호출해 실행하는 함수.   
  
```c++
// 1부터 n까지 합을 계산하는 반복문과 재귀함수
int sum(int n) {
    int ret = 0;
    for (int i = 0; i <= n; i++)
        ret += i;
    return ret;
}
int recursiveSum(int n) {
    if (n == 1) return 1;
    return n + recursiveSum(n - 1);
}
```

In [1]:
def i_sum(n):
    r = 0
    for i in range(1, n + 1):
        r += i
    return r


def r_sum(n):
    if n == 1:
        return 1
    return n + r_sum(n - 1)

print(i_sum(134), r_sum(134))

9045 9045


In [2]:
%timeit i_sum(1345)

75.1 µs ± 3.12 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [3]:
%timeit r_sum(1345)

298 µs ± 13.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


재귀호출이 느리긴 하다.

n개의 조각 중 n이 아니라 1을 빼내는 경우는 다른 문제가 된다!!  
이 예시에서 1을 빼고 나면 2부터 n까지의 합을 구해야 돼서 원래 문제와 다른 형태이다.  
**모든 재귀 함수는 '더이상 쪼개지지 않는' 최소한의 작업에 도달했을 때 답을 곧장 반환하는 조건문을 포함해야 한다.**  
쪼개지지 않는 가장 작은 작업들 => 기저 사례(Base case)

기저 사례를 선택할 때는 존재하는 모든 입력이 항상 기저 사례의 답을 이용해 계산될 수 있도록 신경써야 한다.  

**중첩 반복분 대체하기**  
0번부터 차례대로 번호 매겨진 n개의 원소 중 네 개를 고르는 모든 경우를 출력하는 문제.  
단순히 4중 for문으로 가능하다.  
```c++ 
for (int i = 0; i < n; i++)
    for (int j = i + 1; j < n; j++)
        for (int k = j + 1; k < n; k++)
            for (int l = k + 1; l < n; l++)
                cout << i << " " << j << " " << k << " " << l << '\n';
```

원소가 늘어나거나 입력에 따라 달라져야 하는 경우 이렇게 할 수 없다...  
위 작업을 네 개의 조각으로 나누면 각 조각에서 하나의 원소를 고르면 된다.  
즉 원소를 고르고 남은 원소들을 고르는 작업을 자기 자신을 호출해 떠넘기는 재귀 함수를 작성할 수 있다.  
남은 원소를 고르는 '작업'을 다음과 같은 입력들의 집합으로 정의할 수 있다.
- 원소들의 총 개수
- 더 골라야 할 원소들의 개수 
- 지금까지 고른 원소들의 번호

```c++
void pick(int n, vector<int>& picked, int toPick) {
    // 기저 사례: 더 고를 원소가 없을 때 고른 원소들을 출력
    if (toPick == 0) { printPicked(picked); return; }
    // 고를 수 있는 가장 작은 번호를 계산
    int smallest = picked.empty() ? 0 : picked.back() + 1;
    // 이 단계에서 원소 하나를 고른다.
    for (int next = smallest; next < n; ++next) {
        picked.push_back(next);
        pick(n, picked, toPick - 1);
        picked.pop_back();                 // 다 고르고 나면 마지막것 빼고 다시 고름..
    }
}
```

In [5]:
def pick(n, toPick, picked=[]):
    if toPick == 0:
        print(picked)
        return
    smallest = picked and picked[-1] + 1 or 0
    for i in range(smallest, n):
        picked.append(i)
        pick(n, toPick - 1, picked)
        picked.pop()

In [6]:
pick(5, 3)

[0, 1, 2]
[0, 1, 3]
[0, 1, 4]
[0, 2, 3]
[0, 2, 4]
[0, 3, 4]
[1, 2, 3]
[1, 2, 4]
[1, 3, 4]
[2, 3, 4]


임의의 n개 중 몇 개를 고르든지 사용할 수 있다.

#### 예제: 보글 게임(BOGGLE)
<img src="http://algospot.com/media/judge-attachments/09ee7a6e752f07b0d99b82a010938ab4/boggle.png">

보글(Boggle) 게임은 그림 (a)와 같은 5x5 크기의 알파벳 격자인 
게임판의 한 글자에서 시작해서 펜을 움직이면서 만나는 글자를 그 순서대로 나열하여 만들어지는 영어 단어를 찾아내는 게임입니다.  
펜은 상하좌우, 혹은 대각선으로 인접한 칸으로 이동할 수 있으며 글자를 건너뛸 수는 없습니다.  
지나간 글자를 다시 지나가는 것은 가능하지만, 펜을 이동하지않고 같은 글자를 여러번 쓸 수는 없습니다.  
  
예를 들어 그림의 (b), (c), (d)는 각각 (a)의 격자에서 PRETTY, GIRL, REPEAT을 찾아낸 결과를 보여줍니다.  
  
보글 게임판과 알고 있는 단어들의 목록이 주어질 때, 보글 게임판에서 각 단어를 찾을 수 있는지 여부를 출력하는 프로그램을 작성하세요.  
  
주의: 알고리즘 문제 해결 전략 6장을 읽고 이 문제를 푸시려는 분들은 주의하세요. 6장의 예제 코드는 이 문제를 풀기에는 너무 느립니다.  
6장의 뒷부분과 8장을 참조하세요.  
  
#### 입력
입력의 첫 줄에는 테스트 케이스의 수 C(C <= 50)가 주어집니다.  
각 테스트 케이스의 첫 줄에는 각 5줄에 5글자로 보글 게임판이 주어집니다.  
게임판의 모든 칸은 알파벳 대문자입니다.  
그 다음 줄에는 우리가 알고 있는 단어의 수 N(1 <= N <= 10)이 주어집니다.  
그 후 N줄에는 한 줄에 하나씩 우리가 알고 있는 단어들이 주어집니다.  
각 단어는 알파벳 대문자 1글자 이상 10글자 이하로 구성됩니다.  
  
#### 출력
각 테스트 케이스마다 N줄을 출력합니다.  
각 줄에는 알고 있는 단어를 입력에 주어진 순서대로 출력하고, 해당 단어를 찾을 수 있을 경우 YES, 아닐 경우 NO를 출력합니다.  

예제 입력
```
1
URLPM
XPRET
GIAET
XTNZY
XOQRS
6
PRETTY
GIRL
REPEAT
KARA
PANDORA
GIAZAPX
```
예제 출력
```
PRETTY YES
GIRL YES
REPEAT YES
KARA NO
PANDORA NO
GIAZAPX YES
```

주어진 칸에서 시작해서 특정 단어를 찾을 수 있는지 확인하자.  
다음과 같은 형태의 함수를 작성하면 된다.  
`hasWord(y, x, word)` = 보글 게임판의 (y, x)에서 시작하는 단어 word의존재 여부를 반환

#### 문제의 분할
함수가 하는 일을 가장 자연스럽게 조각내는 방법은 각 글자를 하나의 조각으로 만드는 것.  
함수 호출시 단어 시작위치를 정해 주기 때문에, 문제의 조각들 중 첫 번째 글자에 해당하는 조각을 간단하게 해결할 수 있다.  
시작 위치에 쓰여 있는 글자가 단어의 첫 글자와 다르면 바로 false 리턴..  
이후 원래 단어 word에서 첫 글자를 뗀 단어 word[1..]을 격자에서 찾는다.  
그 시작위치는 인접한 8개 칸 중 하나. 모두 시도하며 답을 찾는다.

#### 기저사례 선택
더 이상의 탐색 없이 간단히 답을 낼 수 있는 다음 경우들을 기저 사례로 선택
1. 위치 (y, x)에 있는 글자가 원하는 단어의 첫 글자가 아닌 경우 항상 실패
2. 1번 경우가 아닐 때, 원하는 단어가 한 글자인 경우 항상 성공
  
두 조건 간의 순서가 바뀌면 안 된다! 

입력이 잘못되거나 범위에서 벗어난 경우도 기저 사례로 택해서 맨 처음에 처리하면 간결해진다.  
위 두 가지 외에 현재 위치가 보글 게임판을 벗어난 경우를 기저 사례로 선택해 처리.

#### 구현
```c++
const int dx[8] = { -1, -1, -1, 1, 1, 1, 0, 0 };
const int dy[8] = { -1, 0, 1, -1, 0, 1, -1, 1 };
// 5x5 보글 게임 판의 해당 위치에서 주어진 단어가 시작하는지 반환
bool hasWord(int y, int x, const string& word) {
    // 기저 사례 1: 시작 위치가 범위 밖이면 무조건 실패
    if (!inRange(y, x)) return false;
    // 기저 사례 2: 첫 글자가 일치하지 않으면 실패
    if (board[y][x] != word[0]) return false;
    // 기저 사례 3: 단어 길이가 1이면 성공
    if (word.size() == 1) return true;
    // 인접한 8칸 검사
    for (int direction = 0; direction < 8; direction++) {
        int nextY = y + dy[direction], nextX = x + dx[direction];
        // 다음 칸이 범위 안에 있는지, 첫 글자는 일치하는지 확인하지 않고 그냥..
        if (hasWord(nextY, nextX, word.substr(1)))
            return true;
    }
    return false;
}
```    

#### 시간 복잡도 분석
완전 탐색 알고리즘의 시간 복잡도 계산은 매우 단순.  
가능한 답 후보들을 모두 만들어 보기 때문에 가능한 후보의 수를 전부 세어 보기만 하면 된다.  
최악의 경우는 답이 아예 존재하지 않을 때.  
```
A로 가득찬 격자에서, AAAAAH를 찾을 수 없지만 알고리즘은 계속 답을 찾으려 함. 
1. 마지막 글자에 도달하기 전에 주변의 모든 칸에 대해 재귀 호출을 함
2. 각 칸에는 최대 여덟 개의 이웃이 있다.
3. 탐색은 단어의 길이 N에 대해 N-1단계 진행.
```
따라서 우리가 검사하는 후보의 수는 최대 $8^{N-1} = O(8^N)$이 된다.

지수적으로 증가하기 때문에 단어의 길이가 짧은 경우만 가능... 길어지면 다른 설계 패러다임을 사용해야 한다.  

#### 완전 탐색 풀이를 위해 필요한 과정
1. 시간 복잡도는 가능한 경우의 수에 정확히 비례. 최대 크기 입력을 가정했을 때 제한 시간 안에 생성할 수 있을지 가늠한다.
2. 가능한 모든 답의 후보를 만드는 과정을 여러 개의 선택으로 나눈다. 각 선택은 답의 후보를 만드는 과정의 한 조각이 된다.
3. 하나의 조각을 선택해 답의 일부를 만들고, 나머지 답을 재귀 호출을 통해 완성한다.
4. 조각이 하나밖에 남지 않은 경우, 혹은 하나도 남지 않은 경우에는 답을 생성했으므로, 이것을 기저 사례로 선택해 처리한다.

**이론적 배경**: 재귀 호출과 부분 문제
1. 주어진 자연수 수열을 정렬하라.
2. {16, 7, 9, 1, 31}을 정렬하라.
  
얼핏 보면 같은 문제라고 할 수 있지만 두 정의 사이에는 큰 차이가 있다.  
전자는 특정 입력을 지정하지 않았고 후자는 지정했다.  
재귀 호출을 논의할 때 '문제'란 항상 수행해야 할 작업과 그 작업을 적용할 자료의 조합을 의미.  
{1, 2, 3}을 정렬하는 문제와 {3, 2, 1}을 정렬하는 문제는 서로 다른 문제다.  
앞의 두 정의 중 엄밀하게 후자만을 문제의 정의라고 할 수 있다.  
보글 게임 문제에서의 문제는  
`게임판에서의 현재 위치 (y, x) 그리고 단어 word가 주어질 때 해당 단어를 이 칸에서부터 시작해 찾을 수 있는가?`로 정이ㅡ된다.  
이를 확인하기 위해 최대 아홉가지 정보를 알아야 한다.
1. 현재 위치 (y, x)에 단어의 첫 글자가 있는가?
2. 윗 칸 (y - 1, x)에서 시작해서, 단어의 나머지 글자들을 찾을 수 있는가?
3. 왼쪽 위 칸 (y - 1, x - 1)에서 시작해서 단어의 나머지 글자들을 찾을 수 있는가?  
  
...  
...

이 중 2번 이후의 항목은 원래 문제에서 한 조각을 떼어냈을 뿐, 형식은 같은 또 다른 문제를 푼 결과.  
문제를 구성하는 조각들 중 하나를 뺐기 때문에 이 문제들은 원래 문제의 일부라고 할 수 있다.  
이런 문제들을 원래 문제의 `부분 문제`라고 한다.

#### 문제: 소풍(PICNIC)

안드로메다 유치원 익스프레스반에서는 다음 주에 율동공원으로 소풍을 갑니다.  
원석 선생님은 소풍 때 학생들을 두 명씩 짝을 지어 행동하게 하려고 합니다.  
그런데 서로 친구가 아닌 학생들끼리 짝을 지어 주면 서로 싸우거나 같이 돌아다니지 않기 때문에,  
항상 서로 친구인 학생들끼리만 짝을 지어 줘야 합니다.  
  
각 학생들의 쌍에 대해 이들이 서로 친구인지 여부가 주어질 때, 학생들을 짝지어줄 수 있는 방법의 수를 계산하는 프로그램을 작성하세요.  
짝이 되는 학생들이 일부만 다르더라도 다른 방법이라고 봅니다. 예를 들어 다음 두 가지 방법은 서로 다른 방법입니다.  
  
(태연,제시카) (써니,티파니) (효연,유리)  
(태연,제시카) (써니,유리) (효연,티파니)  
  
#### 입력
입력의 첫 줄에는 테스트 케이스의 수 C (C <= 50) 가 주어집니다.  
각 테스트 케이스의 첫 줄에는 학생의 수 n (2 <= n <= 10) 과 친구 쌍의 수 m (0 <= m <= n\*(n-1)/2) 이 주어집니다.  
그 다음 줄에 m 개의 정수 쌍으로 서로 친구인 두 학생의 번호가 주어집니다.  
번호는 모두 0 부터 n-1 사이의 정수이고, 같은 쌍은 입력에 두 번 주어지지 않습니다. 학생들의 수는 짝수입니다.  
  
#### 출력
각 테스트 케이스마다 한 줄에 모든 학생을 친구끼리만 짝지어줄 수 있는 방법의 수를 출력합니다.  
  
예제 입력
```
3 
2 1 
0 1 
4 6 
0 1 1 2 2 3 3 0 0 2 1 3 
6 10 
0 1 0 2 1 2 1 3 1 4 2 3 2 4 3 4 3 5 4 5
```
예제 출력
```
1
3
4
```

#### 풀이

**완전 탐색**  
재귀 호출을 이용해 문제를 해결하려면 우선 각 답을 만드는 과정을 여러 개의 조각으로 나눠야 한다.  
전체 문제를 ${n \over 2}$개의 조각으로 나눠서 한 조각마다 두 학생을 짝지어 주는 것으로 하자.  
이때 문제의 형태는 '아직 짝을 찾지 못한 학생들의 명단이 주어질 때 친구끼리 둘씩 짝짓는 경우의 수를 계산하라'가 된다.  
```c++
int n;
bool areFriends[10][10];
// taken[i] = i번째 학생이 짝을 이미 찾았으면 true, 아니면 false
int countPairings(bool taken[10]) {
    // 기저 사례: 모든 학생이 이미 짝을 찾았으면 한 가지 방법을 찾았으니 종료.
    bool finished = true;
    for (int i = 0; i < n; i++) if (!taken[i]) finished = false;
    if (finished) return 1;
    int re = 0;
    // 서로 친구인 두 학생을 찾아 짝을 지어 준다.
    for (int i = 0; i < n; i++) 
        if (!taken[i] && !taken[j] && areFriends[i][j]) {
            taken[i] = taken[j] = true;
            ret += countPairings(taken);
            taken[i] = taken[j] = false;
        }
    return ret;
}
```
    

**중복으로 세는 문제**  
1. 같은 학생 쌍을 두 번 짝지어 준다. (0, 1)과 (1, 0)을 따로 세고 있다.
2. 다른 순서로 학생들을 짝지어 주는 것을 서로 다른 경우로 세고 있다.
    - (0, 1) 후에 (2, 3)을 짝지어 주는 것과 (2, 3)후에 (0, 1)을 짝지어주는 것을 다른 경우로 세고 있다.  
  
이러한 상황을 해결하기 위한 좋은 방법은 `항상 특정 형태를 갖는 답만을 세는 것`이다.  
흔히 사용하는 방법으로 같은 답 중 사전순으로 가장 먼저 오는 답 하나만을 세는 것이 있다.  
이 속성을 강제하기 위해서 각 단계에서 남아 있는 학생들 중 가장 번호가 빠른 학생의 짝을 찾아 주도록 하면 된다.  
이렇게 하면 위의 두 문제 모두 해결됨.  
```c++
int n;
bool areFriends[10][10];
int countPairings(bool taken[10]) {
    // 남은 학생들 중 가장 번호가 빠른 학생을 찾는다.
    int firstFree = -1;
    for (int i = 0; i < n; i++) {
        if (!taken[i]) {
            firstFree = i;
            break;
        }
    }
    // 기저 사례: 모든 학생이 짝을 찾은 경우
    if (firstFree == -1) return 1;
    int ret = 0;
    // 이 학생과 짝지을 학생을 결정.
    for (int pairWith = firstFree + 1; pairWith < n; ++pairWith) {
        if (!takne[pairWith] && areFriends[firstFree][pairWith]) {
        taken[fisrtFree] = taken[pairWith] = true;
        ret += countPairings(taken);
        taken[firstFree] = taken[pairWith] = false;
        }
    }
    return ret;
}
```

전체 문제 풀이 코드.. c++
```c++
#include <iostream>
using namespace std;

int C, n, m, s1, s2;
bool areFriends[10][10];
bool taken[10];

int countPairings(bool taken[10]) {
	// 남은 학생들 중 가장 번호가 빠른 학생을 찾는다.
	int firstFree = -1;
	for (int i = 0; i < n; i++) {
		if (!taken[i]) {
			firstFree = i;
			break;
		}
	}
	// 기저 사례: 모든 학생이 짝을 찾은 경우
	if (firstFree == -1) return 1;
	int ret = 0;
	// 이 학생과 짝지을 학생을 결정.
	for (int pairWith = firstFree + 1; pairWith < n; ++pairWith) {
		if (!taken[pairWith] && areFriends[firstFree][pairWith]) {
			taken[firstFree] = taken[pairWith] = true;
			ret += countPairings(taken);
			taken[firstFree] = taken[pairWith] = false;
		}
	}
	return ret;
}

int main() {
	cin >> C;
	while (C--) {
		for (int j = 0; j < 10; j++) {
			taken[j] = false;
			for (int i = 0; i < 10; i++)
				areFriends[j][i] = false;
		}
			
		cin >> n >> m;
		while (m--) {
			cin >> s1 >> s2;
			areFriends[s1][s2] = areFriends[s2][s1] = true;
		}
		cout << countPairings(taken) << '\n';
	}

	return 0;
}
```

파이썬

In [4]:
def countPairings(taken):
    firstFree = -1
    for i in range(n):
        if not taken[i]:
            firstFree = i
            break
    if firstFree == -1:
        return 1
    
    ret = 0
    for j in range(firstFree + 1, n):
        if not taken[j] and areFriends[firstFree][j]:
            taken[firstFree] = taken[j] = True
            ret += countPairings(taken)
            taken[firstFree] = taken[j] = False
    return ret

for _ in range(int(input())):
    n, m = map(int, input().split())
    taken = [False] * n
    areFriends = [[False] * n for _ in range(n)]
    *friends, = map(int, input().split())
    for i in range(0, 2 * m, 2):
        s1, s2 = friends[i], friends[i + 1]
        areFriends[s1][s2] = areFriends[s2][s1] = True
    print(countPairings(taken))

3
2 1
0 1
1
4 6
0 1 1 2 2 3 3 0 0 2 1 3
3
6 10
0 1 0 2 1 2 1 3 1 4 2 3 2 4 3 4 3 5 4 5
4


#### 답의 수의 상한
모든 답을 생성해 가며 답의 수를 세는 재귀 호출 알고리즘은 답의 수에 정비례 하는 시간이 걸린다.  
따라서 실제로 프로그램을 짜기 전에 답의 수가 얼마나 될지 예측해 보고 모든 답을 만드는 데 시간이 얼마나 오래 걸릴지 확인해야 함.  
이 문제에서 가장 많은 답을 가질 수 있는 입력은 열 명의 학생이 모두 서로 친구인 경우.  
이때 번호가 가장 빠른 학생이 선택할 수 있는 짝은 9명, 그 다음 학생은 7명,..  
최종 답의 개수는 9 x 7 x 5 x 3 x 1 = 945

#### 문제: 게임판 덮기(BOARDCOVER)

<img src="http://algospot.com/media/judge-attachments/2b7bfee35cbec2f4e799bb011ac18f69/03.png">
  
H\*W 크기의 게임판이 있습니다.  
게임판은 검은 칸과 흰 칸으로 구성된 격자 모양을 하고 있는데 이 중 모든 흰 칸을 3칸짜리 L자 모양의 블록으로 덮고 싶습니다.  
이 때 블록들은 자유롭게 회전해서 놓을 수 있지만, 서로 겹치거나, 검은 칸을 덮거나, 게임판 밖으로 나가서는 안 됩니다.  
위 그림은 한 게임판과 이를 덮는 방법을 보여줍니다.  
  
게임판이 주어질 때 이를 덮는 방법의 수를 계산하는 프로그램을 작성하세요.  
  
#### 입력
입력의 첫 줄에는 테스트 케이스의 수 C (C <= 30) 가 주어집니다.  
각 테스트 케이스의 첫 줄에는 2개의 정수 H, W (1 <= H,W <= 20) 가 주어집니다.  
다음 H 줄에 각 W 글자로 게임판의 모양이 주어집니다. # 은 검은 칸, . 는 흰 칸을 나타냅니다.  
입력에 주어지는 게임판에 있는 흰 칸의 수는 50 을 넘지 않습니다.  
  
#### 출력
한 줄에 하나씩 흰 칸을 모두 덮는 방법의 수를 출력합니다.  
  
예제 입력
```
3 
3 7 
#.....# 
#.....# 
##...## 
3 7 
#.....# 
#.....# 
##..### 
8 10 
########## 
#........# 
#........# 
#........# 
#........# 
#........# 
#........# 
########## 
```
예제 출력
```
0
2
1514
```

#### 풀이
일단  흰 칸의 수가 3의 배수가 아닐 경우 무조건 답이 없으니 따로 처리.  
이 외에는 3으로 나눠 필요한 블록 수 N을 얻은 뒤 문제의 답을 생성하는 과정을 N조각으로 나눠서 한 조각에서 한 블록을 내려놓자.  
재귀 함수는 주어진 게임판에 블록을 한 개 내려놓고 남은 흰 칸들을 재귀 호출을 이용해 덮도록 하면 된다.  
  
**중복으로 세는 문제**  
블록을 놓는 순서는 이 문제에서 중요하지 않다.  
이 문제에서도 특정 순서대로 답을 생성하도록 강제해줘야 한다.  
가장 간편한 방법은 재귀 호출의 각 단계마다 아직 빈 칸 중에서 가장 윗 줄, 또 가장 왼쪽에 있는 칸을 덮도록 하는 것.  
빈 칸 중 가장 위, 가장 왼쪽에 있는 칸을 처음 채운다고 가정하기 때문에 그 왼쪽과 위에 있는 칸은 항상 이미 채워져 있다고 가정.  
따라서 각 칸을 채우는 방법은 모두 네 가지 경우가 있다.  
재귀 호출 알고리즘은 빈 칸을 찾은 후 덮을 방법을 하나하나 시도.  
이 방법이 달라질 때마다 서로 다른 배치가 되므로, 우리가 원하는 답은 남은 게임판을 재귀 호출에 넘겨서 얻은 경우의 수를 모두 더한 수.  

**답의 수의 상한**  
알고리즘에서 한 블록을 놓을 때마다 모두 네 가지의 선택지, 최대 16개의 블록.  
따라서 가능한 답의 상한은 $4^{16} = 2^{32}$개.  
매우 많아보이지만 각 단계에서 우리가 선택할 수 있는 블록 배치가 크게 제한됨을 알 수 있다.  
실제 답의 수는 이 상한보다 훨씬 작으리라고 예측할 수 있다.  

#### 구현
```c++
// 주어진 칸을 덮을 수 있는 네 가지 방법
// 블록을 구성하는 세 칸의 상대적 위치 (dy, dx)의 목록
const int coverType[4][3][2] = {    // 4가지 경우, 칸 3개, x, y좌표 차이
    { { 0, 0 }, { 1, 0 }, { 0, 1 } },
    { { 0, 0 }, { 0, 1 }, { 1, 1 } },
    { { 0, 0 }, { 1, 0 }, { 1, 1 } },
    { { 0, 0 }, { 1, 0 }, { 1, -1 } }
};
// board의 (y, x)를 type번 방법으로 덮거나, 덮었던 블록을 없앤다.
// delta = 1이면 덮고, -1이면 덮었던 블록을 없앤다.
// 만약 블록이 제대로 덮이지 않은 경우 (게임판 밖으로 나가거나, 겹치거나, 검은 칸을 덮을 때) false 반환.
bool set(vector<vector<int> >& board, int y, int x, int type, int delta) {
    bool ok = true;
    for (int i = 0; i < 3; i++) {
        const int ny = y + coverType[type][i][0];
        const int nx = x + coverType[type][i][1];
        if (ny < 0 || ny >= board.size() || nx < 0 || nx >= board[0].size()) ok = false;
        else if ((board[ny][nx] += delta) > 1) ok = false;
    }
    return ok;
}
// board의 모든 빈 칸을 덮을 수 있는 방법의 수를 반환
// board[i][j] = 1이미 덮인 칸 혹은 검은 칸
// board[i][j] = 0 아직 덮이지 않은 칸
int cover(vector<vector<int> >& board) {
    // 아직 채우지 못한 칸 중 가장 윗줄 왼쪽에 있는 칸을 찾는다.
    int y = -1, x = -1;
    for (int i = 0; i < board.size(); i++) {
        for (int j = 0; j < board.size(); j++)
            if (board[i][j] == 0) {
                y = i;
                x = j;
                break;
            }
        if (y != -1) break;
    }
    // 기저 사례: 모든 칸을 채웠으면 1을 반환
    if (y == -1) return 1;
    int ret = 0;
    for (int type = 0; type < 4; type++) {
        // 만약 board[y][x]를 type 형태로 덮을 수 있으면 재귀 호출한다.
        if (set(board, y, x, type, 1))
            ret += cover(board);
        // 덮었던 블록을 치운다.
        set(board, y, x, type, -1);
    }
    return ret;
}
```

- 한 칸을 덮는 네 가지 방법을 각각의 코드로 구현하는 것이 아니라 coverType[]배열에 저장했다.  
    - 네 가지 방법에서 새로 채워질 칸들의 상대 좌표 ($\Delta y, \Delta x$)의 목록을 저장
- set()은 delta에 따라 블록을 놓는 역할과 치우는 역할을 같이 할 수 있다. 별도로 할 필요가 없음..
- set()은 해당 위치에 블록을 놓을 수 있는지 여부도 같이 판단함.  
    - 이때 곧장 함수를 종료하는 것이 아니라 마지막까지 함수를 실행한다.
        - 만약 블록을 구성하는 세 칸 중 한 칸에 표시를 한 뒤 두 번째 칸에 이미 블록이 놓여 있다는 것을 발견했다면,
        - 함수를 곧장 종료하면 나중에 덮었던 블록을 치울 때, 이미 있던 블록마저 치워 버리게 된다.
        - set()은 그 자리에 그냥 1을 더함으로써 이 칸에는 두 개의 블록이 겹쳐서 놓여 있다고 표시한다.

In [6]:
coverType = (((0, 0), (1, 0), (0, 1)),
             ((0, 0), (0, 1), (1, 1)),
             ((0, 0), (1, 0), (1, 1)),
             ((0, 0), (1, 0), (1, -1)))

def _set(y, x, t, delta):
    ok = True
    for i in range(3):
        ny, nx = y + coverType[t][i][0], x + coverType[t][i][1]
        if ny < 0 or ny >= H or nx < 0 or nx >= W:
            ok = False
        else:
            board[ny][nx] += delta
            if board[ny][nx] > 1:
                ok = False
    return ok


def cover():
    x = y = -1
    for j in range(H):
        for i in range(W):
            if board[j][i] == 0:
                x, y = i, j
                break
        else:
            continue                     
        break
    if y == -1:
        return 1
    ret = 0
    for t in range(4):
        if _set(y, x, t, 1):
            ret += cover()
        _set(y, x, t, -1)
    return ret
    
    
for _ in range(int(input())):
    H, W = map(int, input().split())
    board = [[0] * W for _ in range(H)]
    n_blank = 0
    for j in range(H):
        row = input()
        for i in range(W):
            if row[i] == '#':
                board[j][i] = 1
            else:
                n_blank += 1

    print(((n_blank % 3) != 0) and '0' or cover())

1
3 7
#.....#
#.....#
##..###
2


### 최적화 문제(Optimization Problem)
지금까지와 달리 문제의 답이 하나가 아니라 여러 개이고, 그 중 어떤 기준에 따라 가장 좋은 답을 찾는 문제.  
n개 원소 중 r개를 순서 없이 골라내는 방법의 수를 계산하는 것 => 최적화 문제 아님  
n개의 사과 중 r개를 골라 무게의 합을 최대화 하는 문제 => 최적화 문제  
여기서도 가장 기초적인 것이 완전 탐색.

#### 예제: 여행하는 외판원 문제(Traveling Salesman Problem, TSP)
어떤 나라에 $n(2 \le n \le 10)$개의 큰 도시가 있다고 하자.  
한 영업 사원이 한 도시에서 출발해 다른 도시들을 전부 한 번씩 방문한 뒤 시작 도시로 돌아오려고 한다.  
각 도시들은 모두 직선 도로로 연결되어 있다.  
영업 사원이 여행해야 할 거리는 어느 순서로 각 도시들을 방문하느냐에 따라 달라진다.  
가능한 모든 경로 중 가장 짧은 경로를 찾으려면??  

**무식하게 풀 수 있을까?**
완전 탐색 첫 단계는 시간 안에 답을 구할 수 있는지 확인.  
시작한 도시로 돌아오는 경로를 찾기 때문에 시작점은 신경 쓰지 않고 무조건 0번 도시에서 출발한다고 해도 된다.  
n - 1개의 도시를 나열하는 방법은 모두 (n - 1)!가지.  
도시가 열 개라면 경로의 수는 9! = 362,880개. 이정도는 가능!

**재귀 호출을 통한 답안 생성**
n개의 도시로 구성된 경로를 n개의 조각으로 나눠, 앞에서부터 도시를 하나씩 추가해 경로를 만들어 가자.  
```
shortestPath(path) = path가 지금까지 만든 경로일 때, 나머지 도시들을 모두 방문하는 경로들 중 가장 짧은 것의 길이 반환
```
```c++
int n; // 도시의 수
double dist[MAX][MAX]; // 두 도시 간의 거리를 저장하는 배열
// path: 지금까지 만든 경로
// visited: 각 도시 방문 여부
// currentLength: 지금까지 만든 경로 길이
// 나머지 도시들을 모두 방문하는 경로들 중 가장 짧은 것의 길이를 반환한다.
double shortestPath(vector<int>& path, vector<bool>& visited, double currentLength) {
    // 기저 사례: 모든 도시를 다 방문했을 때는 시작 도시로 돌아가고 종료한다.
    if (path.size() == n)
        return currentLength + dist[path[0]][path.back()];
    double ret = INF;  // 매우 큰 값으로 초기화
    // 다음 방문할 도시를 전부 시도해 본다.
    for (int next = 0; next < n; ++next) {
        if (visited[next]) continue;
        int here = path.back();
        path.push_back(next);
        visited[next] = true;
        // 나머지 경로를 재귀 호출을 통해 완성하고 가장 짧은 경로의 길이를 얻는다.
        double cand = shortestPath(path, visited, currentLength + dist[here][next]);
        ret = min(ret, cand);
        visited[next] = false;
        path.pop_back();
    }
    return ret;
}
```
각 정점을 방문했는지를 나타내는 불린 값 배열 visited와 현재 경로의 길이 currentLength를 path와 함께 인자로 받고 있다.  
path를 이용해 이 두 가지의 값을 쉽게 계산할 수 있기 때문에 사실 불필요.  
한 가지 정보는 한 가지 형태로만 저장한다는 프로그래밍 원칙에는 어긋남.  
하지만 **매번 각 정점을 방문했는지 확인하는 작업과 현재 경로의 길이를 구하는 작업을 더 효율적으로 수행**할 수 있도록 해준다.

In [11]:
dist = [[0] * 10 for _ in range(10)]
visited = [False] * 10

def shortestPath(path, currentLength):
    if len(path) == n:
        return currentLength + dist[path[0]][path[-1]]
    ret = float('inf')
    for nxt in range(n):
        if visited[nxt]:
            continue
        here = path[-1]
        path.append(nxt)
        visited[nxt] = True
        
        cand = shortestPath(path, currentLength + dist[here][nxt])
        ret = min(ret, cand)
        visited[nxt] = False
        path.pop()
    return ret

#### 문제: 시계 맞추기(CLOCKSYNC)
<img src="http://algospot.com/media/judge-attachments/d3428bd7a9a425b85c9d3c042b674728/clocks.PNG">

그림과 같이 4 x 4 개의 격자 형태로 배치된 16개의 시계가 있다. 이 시계들은 모두 12시, 3시, 6시, 혹은 9시를 가리키고 있다.  
이 시계들이 모두 12시를 가리키도록 바꾸고 싶다.  
  
시계의 시간을 조작하는 유일한 방법은 모두 10개 있는 스위치들을 조작하는 것으로, 각 스위치들은 모두 적게는 3개에서 많게는 5개의 시계에 연결되어 있다.  
한 스위치를 누를 때마다, 해당 스위치와 연결된 시계들의 시간은 3시간씩 앞으로 움직인다.  
스위치들과 그들이 연결된 시계들의 목록은 다음과 같다.  
```
0    0, 1, 2
1    3, 7, 9, 11
2    4, 10, 14, 15
3    0, 4, 5, 6, 7
4    6, 7, 8, 10, 12
5    0, 2, 14, 15
6    3, 14, 15
7    4, 5, 7, 14, 15
8    1, 2, 3, 4, 5
9    3, 4, 5, 9, 13
```
시계들은 맨 윗줄부터, 왼쪽에서 오른쪽으로 순서대로 번호가 매겨졌다고 가정하자.  
시계들이 현재 가리키는 시간들이 주어졌을 때, 모든 시계를 12시로 돌리기 위해 최소한 눌러야 할 스위치의 수를 계산하는 프로그램을 작성하시오.  
  
#### 입력
첫 줄에 테스트 케이스의 개수 C (<= 30) 가 주어진다.   
각 테스트 케이스는 한 줄에 16개의 정수로 주어지며, 각 정수는 0번부터 15번까지 각 시계가 가리키고 있는 시간을 12, 3, 6, 9 중 하나로 표현한다.  
  
#### 출력
각 테스트 케이스당 한 줄을 출력한다. 시계들을 모두 12시로 돌려놓기 위해 눌러야 할 스위치의 최소 수를 출력한다.  
만약 이것이 불가능할 경우 -1 을 출력한다.  
  
예제 입력
```
2
12 6 6 6 6 6 12 12 12 12 12 12 12 12 12 12 
12 9 3 12 6 6 9 3 12 9 12 9 12 12 6 6
```
예제 출력
```
2
9
```

#### 풀이

**문제 변형하기**  
있는 그대로 풀려 하면 꽤나 복잡..  
문제의 특성을 이용해 적절히 단순화하면 완전 탐색으로 해결할 수 있다.

- 스위치를 누르는 순서는 전혀 중요하지 않다.
    - 두 스위치를 누르는 순서를 바꾼다고 해서 결과가 바뀌지 않는다.
    - 각 스위치를 몇 번이나 누를 것인지만 중요
- 시계는 12시간이 지나면 제 자리로 돌아온다!
    - 어떤 스위치건 최대 세 번 이상 누를 일이 없다.
    - 각 스위치를 누르는 횟수는 0에서 3 사이의 정수.
    - 스위치 10개가 있으므로 전체 경우의 수는 $4^{10} = 1,048,576$개. 완전탐색 가능!

**완전 탐색 구현하기**  
문제를 모두 열 조각으로 나눈 후 각 조각에서 한 스위치를 누를 횟수를 정하는 식으로 구현.  
재귀 호출 깊이가 정해져 있기 때문에 10중 for문과 동일하지만 구현, 디버깅이 훨씬 쉽고 편하다.  
```cpp
const int INF = 0000, SWITCHES = 10, CLOCKS = 16;
// linked[i][j] = 'x': i번 스위치와 j번 시계가 연결되어 있다.
// linked[i][j] = '.': i번 스위치와 j번 시계가 연결되어 있지 않다.
const char linked[SWITCHES][CLOCKS+1] = {
// 0123456789012345
    "xxx.............",
    "...x...x.x.x....",
    "....x.....x...xx",
    "x...xxxx........",
    "......xxx.x.x...",
    "x.x...........xx",
    "...x..........xx",
    "....xx.x......xx",
    ".xxxxx..........",
    "...xxx...x...x.."};
// 모든 시계가 12시를 가리키고 있는지 확인.
bool areAligned(const vector<int>& clocks) {
    for (int i = 0; i < 16; i++) {
        if (clocks[i] != 12) return false;
    return true;
        
// swtch번 스위치를 누른다.
void push(vector<int>& clocks, int swtch) {
    for (int clock = 0; clock < CLOCKS; clock++)
        if (linked[swtch][clock] == 'x') {
            clocks[clock] += 3;
            if (clocks[clock] == 15) clocks[clock] = 3;
        }
}
// clocks: 현재 시계들의 상태
// swtch: 이번에 누를 스위치의 번호
// 가 주어질 때, 남은 스위치들을 눌러서 clocks를 12시로 맞출 수 있는 최소 횟수를 반환
// 만약 불가능하다면 INF 이상의 큰 수를 반환
int solve(vector<int>& clocks, int swtch) {
    if (swtch == SWITCHES) return areAligned(clocks) ? 0 : INF;
    // 이 스위치를 0번 누르는 경우부터 세 번 누르는 경우까지를 모두 시도.
    int ret = INF;
    for (int cnt = 0; cnt < 4; cnt++) {
        ret = min(ret, cnt + solve(clocks, swtch + 1));
        push(clocks, swtch);
    }
    // push(clocks, swtch)가 네 번 호출됐으니 clocks는 원래와 같은 상태가 된다.
    return ret;
}
```

### 많이 등장하는 완전 탐색 유형
1. 모든 순열 만들기(Permutation)
    - 서로 다른 N개의 원소를 일렬로 줄 세우기.
    - 가능한 순열의 수는 N!이 되는데, N이 10을 넘어가면 시간 안에 모든 순열을 생성하기 어려움. 다른 방법을 모색해야함.
    - c++은 STL에 포함된 next_permutation()함수 존재..
    - 직접 작성 => 재귀 호출 이용
2. 모든 조합 만들기(Combination)
    - 서로 다른 N개의 원소 중 R개를 순서 없이 골라내기
    - 경우의 수는 이항 계수 ${N \choose R}$로 정의.
3. $2^n$가지 경우의 수 만들기
    - n개의 질문에 대한 답이 예/아니오 중의 하나라고 할 때 존재할 수 있는 답의 모든 조합의 수는 $2^n$ 가지
    - 각 조합을 하나의 n비트 정수로 표현한다고 하면 재귀 호출 없이 1차원 for문 하나로 이 조합들 간단하게 모두 시도할 수 있다.